In [1]:
%matplotlib inline

In [2]:
#import modules
from utils import *
from vgg16 import Vgg16

Using Theano backend.


In [23]:
%pwd

'C:\\Users\\Manuel\\Projects\\deepLearning\\lesson2'

In [24]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/../data/dogscatsredux_scratch'

## Create validation set and sample

In [38]:
%%bash
cd $DATA_HOME_DIR
mkdir -p sample/train
mkdir -p sample/test
mkdir -p sample/valid
mkdir -p sample/results
mkdir -p test/unknown

In [39]:
%cd $DATA_HOME_DIR/train

C:\Users\Manuel\Projects\deepLearning\data\dogscatsredux_scratch\train


In [42]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000):
    os.rename(shuf[i], DATA_HOME_DIR+'/valid/'+shuf[i])

In [43]:
from shutil import copyfile

In [44]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200):
    copyfile(shuf[i], DATA_HOME_DIR+'/sample/train/'+shuf[i])

In [45]:
%cd $DATA_HOME_DIR/valid

C:\Users\Manuel\Projects\deepLearning\data\dogscatsredux_scratch\valid


In [46]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], DATA_HOME_DIR+'/sample/valid/' + shuf[i])

## Rearrange image files into their respective directories

In [66]:
%set_env DATA_HOME=$DATA_HOME_DIR


env: DATA_HOME=C:\Users\Manuel\Projects\deepLearning\lesson2/../data/dogscatsredux_scratch


In [67]:
%%bash
#Divide cat/dog images into separate directories

cd $DATA_HOME/sample/train
mkdir cats
mkdir dogs
mv cat.*.jpg cats/
mv dog.*.jpg dogs/

cd $DATA_HOME/sample/valid
mkdir cats
mkdir dogs
mv cat.*.jpg cats/
mv dog.*.jpg dogs/

cd $DATA_HOME/valid
mkdir cats
mkdir dogs
mv cat.*.jpg cats/
mv dog.*.jpg dogs/

cd $DATA_HOME/train
mkdir cats
mkdir dogs
mv cat.*.jpg cats/
mv dog.*.jpg dogs/

mkdir: cannot create directory 'cats': File exists
mkdir: cannot create directory 'dogs': File exists
mkdir: cannot create directory 'cats': File exists
mkdir: cannot create directory 'dogs': File exists
mkdir: cannot create directory 'cats': File exists
mkdir: cannot create directory 'dogs': File exists
mkdir: cannot create directory 'cats': File exists
mkdir: cannot create directory 'dogs': File exists
mv: cannot stat 'cat.*.jpg': No such file or directory
mv: cannot stat 'dog.*.jpg': No such file or directory


In [68]:
%%bash
cd $DATA_HOME/test/
mv *.jpg unknown/

## Finetuning and Training

In [77]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
#path = DATA_HOME_DIR + '/'
path = DATA_HOME_DIR + '/sample/'
test_path = path + '/test/' #We use all the test data
results_path=path + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

C:\Users\Manuel\Projects\deepLearning\data\dogscatsredux_scratch


In [71]:
vgg = Vgg16()

In [72]:
#Set constants. You can experiment with no_of_epochs to improve the model
batch_size=64
no_of_epochs=3

In [73]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)
vgg.finetune(batches)

#Not sure if we set this for all fits
vgg.model.optimizer.lr = 0.01

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [75]:
#Notice we are passing in the validation dataset to the fit() method
#For each epoch we test our model against the validation set
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print("Running epoch: %d" % epoch)
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
print("Completed %s fit operations" % no_of_epochs)

Running epoch: 0
Epoch 1/1
200/200 [==============================] - 107s - loss: 3.4856 - acc: 0.6050 - val_loss: 0.3379 - val_acc: 0.9800
Running epoch: 1
Epoch 1/1
200/200 [==============================] - 110s - loss: 0.2953 - acc: 0.9700 - val_loss: 0.3270 - val_acc: 0.9800
Running epoch: 2
Epoch 1/1
200/200 [==============================] - 107s - loss: 0.6218 - acc: 0.9350 - val_loss: 0.3224 - val_acc: 0.9800
Completed 3 fit operations


## Generate Predictions

In [79]:
batches, preds = vgg.test(test_path, batch_size= batch_size*2)

Found 32 images belonging to 1 classes.


In [80]:
filenames = batches.filenames

In [81]:
#Save our test results arrays so we can use them again later
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)

## Submit Predictions to Kaggle!

In [82]:
preds = load_array(results_path + 'test_preds.dat')
filenames = load_array(results_path + 'filenames.dat')

In [83]:
#Grab the dog prediction column
isdog = preds[:,1]

In [84]:
#So to play it safe, we use a sneaky trick to round down our edge predictions
#Swap all ones with .95 and all zeros with .05
isdog = isdog.clip(min=0.05, max=0.95)

In [85]:
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [88]:
subm = np.stack([ids,isdog],axis=1)
subm[:5]

array([[  1.  ,   0.95],
       [ 10.  ,   0.05],
       [ 11.  ,   0.05],
       [ 12.  ,   0.95],
       [ 13.  ,   0.05]])

In [89]:
%cd $DATA_HOME_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

C:\Users\Manuel\Projects\deepLearning\data\dogscatsredux_scratch


In [93]:
from IPython.display import FileLink
%cd $LESSON_HOME_DIR
FileLink(DATA_HOME_DIR+'/'+submission_file_name)

C:\Users\Manuel\Projects\deepLearning\lesson2


C:\Users\Manuel\Projects\deepLearning\data\dogscatsredux_scratch\submission1.csv